# BingSearch using Kernel 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documentation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview)


Prepare a Semantic Kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.17.230626.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using Microsoft.SemanticKernel.TemplateEngine;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAITextCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Add the following namespaces to use the Bing Search connector.

In [ ]:
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;

Enter your Bing Search Key in BING_KEY using `InteractiveKernel` method as introduced in [`.NET Interactive`](https://github.com/dotnet/interactive/blob/main/docs/kernels-overview.md)

In [ ]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string BING_KEY = await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search Key");

Below are some examples on how [`WebSearchEngineSkill`](../../../dotnet/src/Skills/Skills.Web/WebSearchEngineSkill.cs) can be used in SK. 

In [ ]:
private static async Task Example1Async(IKernel kernel)
{
        Console.WriteLine("Example 1");

        // Run 
        var question = "What is quantum tunnelling";
        var bingResult = await kernel.Func("bing", "search").InvokeAsync(question);

        Console.WriteLine(question);
        Console.WriteLine("----");
        Console.WriteLine(bingResult);
        Console.WriteLine();

        /* OUTPUT:

            What is quantum tunnelling
            ----
            In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical
            phenomenon in which an object such as an electron or atom passes through a potential energy 
            barrier that, according to classical mechanics, the object does not have sufficient energy to
            enter or surmount.
            
       */
}

In [ ]:
private static async Task Example2Async(IKernel kernel)
{
    Console.WriteLine("Example 2");

    //The following function only works in interactive notebooks
    string question = await InteractiveKernel.GetInputAsync("Please ask your question"); 

    var bingResult = await kernel.Func("bing", "search").InvokeAsync(question);

    Console.WriteLine(bingResult);
}

Just uncomment the examples to run them in the following code:

In [ ]:
// Load Bing skill
using (var bingConnector = new BingConnector(BING_KEY))
{
    kernel.ImportSkill(new WebSearchEngineSkill(bingConnector), "bing");

    //await Example1Async(kernel);
    //await Example2Async(kernel);
}